In [1]:
from ontolearn.concept_learner import NCES
from ontolearn.knowledge_base import KnowledgeBase
from owlapy.parser import DLSyntaxParser
from owlapy.render import DLSyntaxObjectRenderer
from ontolearn.metrics import F1, Accuracy, Precision, Recall

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
def quality(solution, pos, neg):
    f1 = F1().score2; accuracy = Accuracy().score2; precision = Precision().score2; recall = Recall().score2
    tp=len(pos.intersection(set(KB.individuals(solution))))
    fn=len((set(KB.individuals(solution))-pos).difference(neg))
    fp=len((set(KB.individuals(solution))-neg).difference(pos))
    tn=len((set(KB.individuals(solution))-pos).intersection(neg))
    print("Accuracy: {}%".format(100*accuracy(tp, fn, fp, tn)[-1]))
    print("Precision: {}%".format(100*precision(tp, fn, fp, tn)[-1]))
    print("Recall: {}%".format(100*recall(tp, fn, fp, tn)[-1]))
    print("F1: {}%".format(100*f1(tp, fn, fp, tn)[-1]))

In [3]:
nces = NCES(knowledge_base_path="./NCESData/family/family.owl", learner_name="SetTransformer",
     path_of_embeddings="./NCESData/family/embeddings/ConEx_entity_embeddings.csv", load_pretrained=True, max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32, pretrained_model_name="SetTransformer")



 Loaded pretrained model! 



In [4]:
KB = KnowledgeBase(path=nces.knowledge_base_path)

In [5]:
dl_syntax_renderer = DLSyntaxObjectRenderer()

In [6]:
atomic_classes = [dl_syntax_renderer.render(a) for a in KB.ontology().classes_in_signature()]

In [7]:
atomic_classes

['Brother',
 'Male',
 'PersonWithASibling',
 'Child',
 'Person',
 'Daughter',
 'Female',
 'Father',
 'Parent',
 'Grandchild',
 'Granddaughter',
 'Grandfather',
 'Grandparent',
 'Grandmother',
 'Grandson',
 'Mother',
 'Sister',
 'Son']

In [8]:
dl_parser = DLSyntaxParser(nces.kb_namespace)

In [9]:
brother = dl_parser.parse('Brother')
daughter = dl_parser.parse('Daughter')

#### Input examples can be sets or lists

In [10]:
pos = set(KB.individuals(brother)).union(set(KB.individuals(daughter)))

In [11]:
neg = set(KB.individuals())-set(pos)

#### Prediction with SetTransformer (default model)

In [12]:
concept = nces.fit(pos, neg)

Prediction:  Son ⊔ Daughter ⊔ PersonWithASibling


In [13]:
quality(concept, pos, neg)

Accuracy: 100.0%
Precision: 100.0%
Recall: 100.0%
F1: 100.0%


### Ensemble prediction

In [14]:
nces.pretrained_model_name = ['SetTransformer','GRU','LSTM']
nces.refresh()
concept = nces.fit(pos, neg)



 Loaded pretrained model! 



 Loaded pretrained model! 



 Loaded pretrained model! 

Prediction:  Brother ⊔ Daughter


In [15]:
quality(concept, pos, neg)

Accuracy: 100.0%
Precision: 100.0%
Recall: 100.0%
F1: 100.0%


#### Complex learning problems, potentially without an exact solution

In [16]:
import random
random.seed(42)

In [17]:
all_individuals = set(KB.individuals())
pos = set(random.sample(list(all_individuals), 50))
neg = set(random.sample(list(all_individuals-pos), 50))

In [18]:
nces.pretrained_model_name

['SetTransformer', 'GRU', 'LSTM']

In [19]:
concept = nces.fit(pos, neg)

Prediction:  Person ⊓ (∀ married.(PersonWithASibling ⊔ (∀ hasChild.(¬Grandson))))


In [20]:
quality(concept, pos, neg)

Accuracy: 31.330000000000002%
Precision: 31.624000000000002%
Recall: 31.624000000000002%
F1: 31.624000000000002%
